# Семинар 12: парсинг и немного пиратства


# 1. Почитать

- азы всех азов: [статейка с хабра](https://habr.com/ru/company/ods/blog/346632/) 
- а парсинг это законно? [статейка с хабра](https://habr.com/ru/post/545818/)
- telegram habr bot как идея для вашего проекта: [статейка с хабра](https://habr.com/ru/post/686174/) 


## Зачем собирать данные автоматически? 

- В общем случае чтобы вытащить информацию, которую собирать руками очень долго (данные о курсах валют, новостные ленты, таблицы с показателями)
- Написать классного телеграм бота с доставанием нужной вам информации
- Забирать данные конкурентов, например, следить за ценами конкурентов и подстраивать свои цены под них
- Собирать тексты, чтобы анализировать их с точки зрения лингвистики или обучать модели
- ... и вы наверняка вспомните случаи когда парсинг вам очень помог бы

<br>

<br>

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/aaaaaa.png" width="500"> 

## Что такое HTML? 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта.

Давайте попробуем зайти на любую страницу, открыть ее код и немного изменить заголовок или текст новостей. Теперь становится понятней откуда берутся многие фейки.

Понятное дело, что вы так меняете html-страничку только на своём компьютере (клиенте). На сервере остаётся исходная версия. Когда вы обновите страничку все правки исчезнут. 

HTML-страница это ни что иное, как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега есть свои потомки (дети) — те теги, которые вложены в него и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:


````
<html>
<head> Заголовок </head>
<body>
    <div>
        Первый кусок текста со своими свойствами
    </div>
    <div>
        Второй кусок текста
            <b>
                Третий, жирный кусок
            </b>
    </div>
    Четвёртый кусок текста
</body>
</html>
````

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tree.png" width="450"> 

## 2. Vanilla parser: Качаем цены на книги

* Хотим собрать [цены на книги](http://books.toscrape.com)
* Руками долго, напишем код на петухоне

Доступ к веб-станицам позволяет получать модуль requests. Подгрузим его. Если у вас не установлен этот модуль, то придётся напрячься и установить:  `pip install requests`.

In [ ]:
!pip install requests >> None # >> None во избежание кучи строк установки библиотеки

In [ ]:
import requests  

url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response

Благословенный 200 ответ - соединение установлено и данные получены, всё чудесно! Если попытаться перейти на несуществующую страницу, то можно получить, например, знаменитую ошибку 404.

In [ ]:
requests.get('http://books.toscrape.com/big_scholarship')

Внутри response лежит html-разметка странички, которую мы парсим. 

In [ ]:
response.text[:1000]

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, прекрасный суп.

<img align="center" src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/alisa.jpg" height="200" width="200"> 

Пакет **[`bs4`](https://www.crummy.com/software/BeautifulSoup/)**, a.k.a **BeautifulSoup** был назван в честь стишка про красивый суп из Алисы в стране чудес. Эта совершенно волшебная библиотека, которая из сырого и необработанного HTML (или XML) кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть однажды, а потом будет проще.

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')

Внутри переменной `tree` теперь лежит дерево из тегов, по которому мы можем совершенно спокойно бродить. 

In [ ]:
tree.title.text.strip()

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [ ]:
tree.html.head.title.text

С текстом можно работать классическими питоновскими методами. Например, можно избавиться от лишних отступов.

In [ ]:
tree.html.head.title.text.strip()

Более того, зная адрес элемента, мы сразу можем найти его. Например, вот так в коде страницы мы можем найти где именно для каждой книги лежит основная информация. Видно, что она находится внутри тега `article`, для которого прописан класс `product_pod` (грубо говоря, в html класс задаёт оформление соотвествующего кусочка страницы). 

Вытащим инфу о книге из этого тега. 

In [ ]:
books = tree.find_all('article', {'class' : 'product_pod'})
books[0]

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём.

In [ ]:
type(books[0])

In [ ]:
books[0].find('p', {'class': 'price_color'}).text

Обратите внимание, что для поиска есть как минимум два метода: `find` и `find_all`. Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый. Чтобы найти все элементы с таким адресом, нужно использовать метод `find_all`. На выход будет выдан список.

Кроме содержимого у тегов часто есть атрибуты. Например, у названия книги есть атрибуты `title` и `href`: 

In [ ]:
books[0].h3

Их тоже можно вытащить.

In [ ]:
books[0].h3.a.get('href')

In [ ]:
books[0].h3.a.get('title')

А ещё по этим атрибутам можно искать интересующие нас кусочки страницы. 

In [ ]:
tree.find_all('a', {'title': 'A Light in the Attic'})

Собственно говоря, это всё. 

Обратите внимание, что на сайте все книги лежат на разных страничках. Если попробовать потыкать их, можно заметить, что в ссылке будет меняться атрибут `page`. Значит, если мы хотим собрать все книги, надо создать кучу ссылок с разным `page` внутри цикла. Когда качаешь данные с более сложных сайтов, в ссылке часто есть огромное количество атрибутов, которые регулируют выдачу.


Давайте запишем весь код для сбора книг в виде функции. На вход она будет принимать номер странички, которую надо скачать. 

In [ ]:
def get_page(p):
    
    # изготовили ссылку
    url = f'http://books.toscrape.com/catalogue/page-{p}.html'
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево 
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # нашли в нём всё самое интересное
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    infa = [ ]
    
    for book in books:
        infa.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': book.h3.a.get('href'),
                     'title': book.h3.a.get('title')})
                     
    return infa

Осталось только пройтись по всем страничкам от page-1 до page-50 циклом и данные у нас в кармане. 

In [ ]:
from tqdm import tqdm # удобная штука для отрисовки прогресс бара

infa = []

for p in tqdm(range(1,10)):
    infa.extend(get_page(p))

In [ ]:
infa

Когда случаются такие объемы информации их зачастую удобнее воспринимать в виде таблички. В питоне для этого придумали целую библиотеку 

__Pandas (Python Data Analysis Library)__ — библиотека языка Python для удобных обработки и анализа данных.

In [ ]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape) # размеры нашего датафрейма (число_строк, число_столбцов)
df.head() # первые N строк (N=5 по дефолту, но в скобках можно указать нужное число строк df.head(5))

Попробуем немного поанализировать данные

In [ ]:
df['price_float'] = df['price'].apply(lambda x: float(x.split('£')[1])) # приведем к числовому формату
print(df.shape)
df.head()

In [ ]:
df.info() # получим общее описание

In [ ]:
df.describe() # работает с числовыми переменными

In [ ]:
print('Average price is:', df.price_float.mean())
print('Min price is:', df.price_float.min())
print('Max price is:', df.price_float.max())

In [ ]:
df[df.price_float < 12].title.values

In [ ]:
df[df.title == 'The Project'].href.item()

In [ ]:
# создадим новую колонку полных ссылок
df['full_link'] = 'http://books.toscrape.com/catalogue/' + df['href']

In [ ]:
df.head(3)

Кстати говоря, если перейти по ссылке в саму книгу, там о ней будет куча дополнительной информации. Можно пройтись по всем ссылкам и выкачать себе по ним дополнительную информацию.

Если на странице, которую вы спарсили, среди тэгов `<tr>` и `<td>` прячется таблица, чаще всего можно забрать её себе без написания цикла, который будет перебирать все стобцы и строки. Поможет в этом `pd.read_html`. Например, вот так можно забрать себе [табличку с сайта ЦБ](https://cbr.ru/currency_base/daily/) 

In [ ]:
import pandas as pd

df = pd.read_html('https://cbr.ru/currency_base/daily/')[0]
df.head()

Команда пытается собрать в массив все таблички c веб-страницы. Если хочется, можно сначала через bs4 найти нужную таблицу, а потом уже распарсить её:

In [ ]:
resp = requests.get('https://cbr.ru/currency_base/daily/')
tree = BeautifulSoup(resp.content, 'html.parser')

# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]

# распарсили её
df = pd.read_html(str(table))[0]
df.head()

А если вам интересно больше о работе в pandas, то можно почитать тут:
- [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html)
- [Pandas intro](https://khashtamov.com/ru/pandas-introduction/)
- [Exploratory data analysis (EDA) using pandas intro](https://habr.com/ru/company/ruvds/blog/494720/)
- [Pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html)

# 3. Selenium

Это инструмент для роботизированного управления браузером. Для его коректной работы нужно скачать драйвер: [для хрома](https://sites.google.com/a/chromium.org/chromedriver/downloads) или [для фаерфокса.](https://github.com/mozilla/geckodriver/releases) 

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# можно установить или указать путь если вы уже скачали драйвер на комп
# https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/
driver = webdriver.Chrome(ChromeDriverManager().install())

Передадим ссылку для перехода

In [ ]:
ref = 'http://google.com'
driver.get(ref)

Найдем место для ввода текста и установим туда курсор:

In [ ]:
stroka = driver.find_element_by_name("q")
stroka.click()

Загуглим скандалы вышки:

In [ ]:
stroka.send_keys('матфак вшэ')

In [ ]:
# находим кнопку для гугления и жмём её
button = driver.find_element_by_name('btnK')
button.click()

Заберем станицу в суп и возьмем все заголовки:

In [ ]:
bs = BeautifulSoup(driver.page_source)

dirty_headings = bs.find_all('h3',attrs={'class':'LC20lb MBeuO DKV0Md'})
clean_headings = [title.text for title in dirty_headings]
clean_headings

In [ ]:
driver.close() # не забываем закрыть браузер

Как еще искать элементы:

- навести в коде страницы на нужный элемент и выбрать `Copy Xpath`
- использовать код `driver.find_element_by_xpath('_____').click()`
- гуглить все что хотите

Селениум можно настроить так, чтобы физически браузер не открывался.

In [ ]:
# https://www.selenium.dev/selenium/docs/api/javascript/module/selenium-webdriver/chrome_exports_Options.html
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

ref = 'http://google.com'
driver.get(ref)

driver.close()

# 3. Лайфхаки если сервер разозлился

* Вы решили собрать себе немного данных 
* Сервер не в восторге от ковровой бомбардировки автоматическими запросами 
* Error 403, 404, 504, $\ldots$ 
* Капча, требования зарегистрироваться
* Заботливые сообщения, что с вашего устройства обнаружен подозрительный трафик

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/doge.jpg" width="450"> 

## а) быть терпеливым 

* Слишком частые запросы раздражают сервер
* Ставьте между ними временные задержки 

In [ ]:
import time
time.sleep(3) # и пусть весь мир подождёт 3 секунды

## б) быть похожим на человека


Запрос нормального человека через браузер выглядит так: 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/browser_get.png" width="600"> 
    
С ним на сервер попадает куча информации! Запрос от питона выглядит так: 


<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/python_get.jpg" width="250"> 
 
Заметили разницу?  Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [fake-useragent.](https://pypi.org/project/fake-useragent/) При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [ ]:
from fake_useragent import UserAgent
UserAgent().chrome

Например, https://knowyourmeme.com/ не захочет пускать к себе python и выдаст ошибку 403. Она выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать.

In [ ]:
url = 'https://knowyourmeme.com/'

response = requests.get(url)
response

А если сгенерировать User-Agent, вопросов у сервера не возникнет. 

In [ ]:
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response

__Другой пример:__ если захотите спарсить ЦИАН, он начнет вам выдавать капчу. Один из вариантов обхода: менять ip через тор. Однако на практически каждый запрос из-под тора, ЦИАН будет выдавать капчу. Если добавить в запрос `User_Agent`, то капча будет вылезать намного реже. 

## в) уходить глубже 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tor.jpg" width="600"> 

Можно попытаться обходить злые сервера через тор. Есть аж несколько способов, но мы про это говорить не будем. Лучше подробно почитать в нашей статье на Хабре. Ссылка на неё в конце тетрадки. Ещё в самом начале была. А ещё в середине [наверняка есть.](https://habr.com/ru/company/ods/blog/346632/)

## Совместить всё? 

1. Начните с малого 
2. Если продолжает банить, накидывайте новые примочки
3. Каждая новая примочка бьёт по скорости 
4. [Разные примочки для requests](http://docs.python-requests.org/en/v0.10.6/user/advanced/)

# 4. Хитрости: 

### Хитрость 1:  Не стесняйтесь пользоваться `try-except`

Эта конструкция позволяет питону в случае ошибки сделать что-нибудь другое либо проигнорировать её. Например, мы хотим найти логарифм от всех чисел из списка: 

In [ ]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    print(log(item))

У нас не выходит, так как логарифм от отрицательных чисел не берётся. Чтобы код не падал при возникновении ошибки, мы можем его немного изменить: 

In [ ]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    try:
        print(log(item))  # попробуй взять логарифм
    except:
        print('я не смог') # если не вышло, сознайся и работай дальше

__Как это использовать при парсинге?__  Интернет создаёт человек. У многих людей руки очень кривые. Предположим, что мы на ночь поставили парсер скачивать цены, он отработал час и упал из-за того, что на како-нибудь одной странице были криво проставлены теги, либо вылезло какое-то редкое поле, либо вылезли какие-то артефакты от старой версии сайта, которые не были учтены в нашем парсере. Гораздо лучше, чтобы код проигнорировал эту ошибку и продолжил работать дальше. 

###  Ещё хитрости: 

* __Сохраняйте то, что парсите по мере скачки!__ Прямо внутрь цикла запихните код, который сохраняет файл! 
* Когда код упал в середине списка для скачки, не обязательно запускать его с самого начала. Просто сохраните тот кусок, который уже скачался и дозапустите код с места падения.
* Засовывать цикл для обхода ссылок внутрь функции - не самая хорошая идея. Предположим, что надо обойти $100$ ссылок. Функция должна вернуть на выход объекты, которые скачались по всему этому добру. Она берёт и падает на $50$ объекте. Конечно же то, что уже было скачано, функция не возвращает. Всё, что вы накачали - вы теряете. Надо запускать заново. Почему? Потому что внутри функции своё пространство имён. Если бы вы делали это циклом влоб, то можно было бы сохранить первые $50$ объектов, которые уже лежат внутри листа, а потом продолжить скачку. 
* Можно ориентироваться на html-страничке с помощью `xpath`. Он предназначен для того, чтобы внутри html-странички можно было быстро находить какие-то элементы. [Подробнее можно почитать тут.](https://devhints.io/xpath)
* Не ленитесь листать документацию. Из неё можно узнать много полезных штук. 

# 6. Почиташки

* [Парсим мемы в python](https://habr.com/ru/company/ods/blog/346632/) - подробная статья на Хабре, по которой можно научиться ... парсить (ВНЕЗАПНО) 
* [Тетрадки Ильи Щурова](https://github.com/ischurov/pythonhse) про python для анализа данных. В [лекции 9](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%209.ipynb) и [лекции 10](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%2010.ipynb) есть про парсеры. 
* [Книга про парсинг](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) на случай если вам совсем скучно и хочется почитать что-то длинное и на английском
* [Продвинутое использование requests](https://2.python-requests.org/en/master/user/advanced/)
* [Перевод документации по selenium на русский на хабре](https://habr.com/ru/post/248559/)


* [Страничка с парсерами одного из семинаристов,](https://fulyankin.github.io/Parsers/) на ней много недописанного и кривого, но есть и интересное: 
    * [Более подробно про selenium](https://nbviewer.jupyter.org/github/FUlyankin/Parsers/blob/master/sems/3_Selenium_and_Tor/4.1%20Selenium%20.ipynb)
    * [Немного устаревший гайд по парсинг вконтакте](https://nbviewer.jupyter.org/github/FUlyankin/ekanam_grand_research/blob/master/0.%20vk_parser_tutorial.ipynb)
    * [Немного устаревший гайд про google maps](https://nbviewer.jupyter.org/github/FUlyankin/Parsers/blob/master/Parsers%20/Google_maps_API.ipynb)

